# Generate object(sign) and annotations on draw

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import os, sys
from glob import glob
from matplotlib import pyplot

from PIL import Image
from PIL import EpsImagePlugin
import math

from PIL import ImageFilter
import numpy as np

from lxml import etree
import xml.etree.ElementTree as xml
import random
import cv2

from sklearn.cluster import KMeans

# preprocessing (Test)

In [98]:
im = cv2.imread('./data/dataset/1-f_pan.png',cv2.IMREAD_GRAYSCALE)
cv2.imwrite('./data/Gray.png', im)

ret, imGray = cv2.threshold(im,150,255, cv2.THRESH_BINARY_INV)
cv2.imwrite('./data/testGray.png', imGray)

True

In [185]:
im = Image.open('./data/TTEST.png')
    
data = np.array(im)


r1, g1, b1 = 255,255,128  # Original value
r2, g2, b2 = 255,0,0 # Value that we want to replace it with

red, green, blue = data[:,:,0], data[:,:,1], data[:,:,2]
mask = (red == r1) & (green == g1) & (blue == b1)
data[:,:,:3][mask] = [r2, g2, b2]

im = Image.fromarray(data)    
imDir = './data/TTEST22.png'
im.save(imDir)

In [20]:
img =  cv2.imread('./DT_150.png')


kernel = np.ones((20, 20), np.uint8)
result4 = cv2.erode(img, kernel, iterations = 1)

kernel = np.ones((30, 30), np.uint8)
result1 = cv2.erode(img, kernel, iterations = 1)


kernel = np.ones((50, 50), np.uint8)
result2 = cv2.erode(img, kernel, iterations = 1)

kernel = np.ones((70, 70), np.uint8)
result3 = cv2.erode(img, kernel, iterations = 1)


cv2.imwrite('./TTEST_erode20.png',  result4) 
cv2.imwrite('./TTEST_erode30.png',  result1) 
cv2.imwrite('./TTEST_erode50.png',  result2) 
cv2.imwrite('./TTEST_erode70.png',  result3) 

True

In [108]:
img =  cv2.imread('./TTEST_erode50.png')

kernel = np.ones((15, 15), np.uint8)
result = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)


cv2.imwrite('./TTEST_erode50_dliate.png',  result ) 


True

In [105]:
img =  cv2.imread('./DT_150.png')

kernel = np.ones((7, 7), np.uint8)
result = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)


cv2.imwrite('./TTEST_dilate.png',  result ) 

True

----------------------------------

# Preprocessing

In [87]:
sign_data = glob('./data/backGround/sign/*.png')
# preprocessing (Sign)

for s in sign_data:
    img = cv2.imread(s)
    kernel = np.ones((10, 10), np.uint8)
    img = cv2.erode(img, kernel, iterations = 1)
    
    cv2.imwrite(s, img)  
    

In [88]:
size = 20, 20

for s in sign_data:
    im = Image.open(s)
    im.thumbnail(size, Image.ANTIALIAS)
    im = im.convert("RGBA")
    datas = im.getdata()
    newData = []
    for item in datas:
        if item[0] > 200 and item[1] > 200 and item[2] > 200:
            newData.append((item[0], item[1], item[2], 0))
        else:
            newData.append(item)

    im.putdata(newData)
    im.save(s)

# Change color of sign

In [90]:
def ChangeColor(inputImg,r2,g2,b2):
    
    im = Image.open(inputImg)
    
    im = np.array(im)
    ret, im = cv2.threshold(im, 247, 255, cv2.THRESH_TOZERO)
    data = im
    
    
    imName = inputImg.split('sign/')[1]
    imName = imName.split('.png')[0]    

    r1, g1, b1 = 0, 0, 0 # Original value
    r2, g2, b2 = r2,g2,b2 # Value that we want to replace it with

    red, green, blue = data[:,:,0], data[:,:,1], data[:,:,2]
    mask = (red == r1) & (green == g1) & (blue == b1)
    data[:,:,:3][mask] = [r2, g2, b2]

    im = Image.fromarray(data)    
    imDir = './data/backGround/New/sign/'+imName+'_r'+str(r2)+'g'+str(g2)+'b'+str(b2)+'.png'
    im.save(imDir)
    
    return imDir

#  Make n-Sign on Draw with loaction(x,y) information 

In [11]:
sign_data = glob('./data/backGround/sign/*.png')
sign_data 

['./data/backGround/sign/DT_002.png',
 './data/backGround/sign/DT_001.png',
 './data/backGround/sign/SM_142.png',
 './data/backGround/sign/DT_143.png',
 './data/backGround/sign/SM_126.png',
 './data/backGround/sign/SM_100.png',
 './data/backGround/sign/DT_150.png',
 './data/backGround/sign/MD_019.png',
 './data/backGround/sign/DT_131.png',
 './data/backGround/sign/DT_127.png',
 './data/backGround/sign/DT_177.png']

In [13]:
draw_data = glob('./data/backGround/draw/*.png')

for i in draw_data:
    im = cv2.imread(i)
    ret, imGray = cv2.threshold(im,150,255, cv2.THRESH_BINARY_INV)

    gray = cv2.cvtColor(imGray, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(i, gray)

In [89]:
dr_fileName = './data/backGround/draw/10f.png'

l_img =  cv2.imread(dr_fileName)
h,w,d = l_img.shape
    
sign_data = glob('./data/backGround/sign/*.png')

signList=[]
# set a color RGB
r2,g2,b2 = 255, 0, 0

# set an amoumnt of sign #

DT_002_amount = 2
DT_001_amount = 2
SM_142_amount = 2
DT_143_amount = 2
SM_126_amount = 2
SM_100_amount = 2
DT_150_amount = 2
MD_019_amount = 2
DT_131_amount = 2
DT_127_amount = 2
DT_177_amount = 2


signAmount = [DT_002_amount,DT_001_amount, SM_142_amount, DT_143_amount, SM_126_amount,SM_100_amount, DT_150_amount, MD_019_amount, DT_131_amount, DT_127_amount, DT_177_amount]
total = sum(signAmount)


new_fileName  =  'GEN_'+str(int(random.random()*100000))+'.png'
new_dirFileName = './data/backGround/New/draw/'+ new_fileName


root = etree.Element("annotation")

folder = etree.SubElement(root, "folder").text ='draw'  
filename =  etree.SubElement(root, "filename").text = new_fileName    
path =  etree.SubElement(root, "path").text = new_dirFileName

source =  etree.SubElement(root, "source")
database =  etree.SubElement(source, "database").text= 'Unknown'     
    
size =  etree.SubElement(root, "size")
width =  etree.SubElement(size, "width").text = str(w)
height =  etree.SubElement(size, "height").text = str(h)
depth =  etree.SubElement(size, "depth").text = str(d)

segmented =  etree.SubElement(root, "segmented").text = '0'


print(total) # 생성할 기호의 총합
print(new_dirFileName) #생성될 도면의 경로


xmin = random.sample(range(50, w), total)
ymin = random.sample(range(50, h), total)
    
location = list(zip(xmin,ymin))


for i, j in zip(sign_data , signAmount):
    
    signDir = i
    clssName = signDir.split('/sign')[1]
    clssName = clssName.split('.png')[0]
    
    signList.append(clssName)

    imDir = ChangeColor(signDir,r2,g2,b2)
      
    s_img =  cv2.imread(imDir,-1)     
    alpha_s = s_img[:, :, 3] / 255.0
    alpha_l = 1.0 - alpha_s 
        
            
    for k in range(0,j):
    
        x1, y1 = location.pop()
        x2 = x1 + s_img.shape[1]
        y2 = y1 + s_img.shape[0]
               
        print(x1, x2, y1, y2)
        
        obj =  etree.SubElement(root, "object")
        name =   etree.SubElement(obj, "name").text = clssName
        poset =   etree.SubElement(obj, "pose").text = 'Unspecified'
        truncated =   etree.SubElement(obj, "truncated").text = '0'
        difficult =   etree.SubElement(obj, "difficult").text = '0'

        bndbox = etree.SubElement(obj, "bndbox")
        xmin =  etree.SubElement(bndbox, "xmin").text = str(x1)
        ymin =  etree.SubElement(bndbox, "ymin").text = str(y1)
        xmax =  etree.SubElement(bndbox, "xmax").text = str(x2)
        ymax =  etree.SubElement(bndbox, "ymax").text = str(y2)
        
        for c in range(0, 3):
            l_img[y1:y2, x1:x2, c] = (alpha_s * s_img[:, :, c] + alpha_l * l_img[y1:y2, x1:x2, c])
            
cv2.imwrite(new_dirFileName, l_img)        
tree = etree.ElementTree(root)
 
with open('./data/backGround/annotations/'+'GEN_'+str(int(random.random()*100000))+'xml', "bw") as fh:
    tree.write(fh)
    
      

22
./data/backGround/New/draw/GEN_78609.png
483 503 270 290
1352 1372 340 360
930 950 409 429
798 818 547 567
516 536 344 359
946 966 863 878
1556 1576 568 588
1538 1558 779 799
657 677 776 796
920 940 974 994
1234 1254 283 294
164 184 787 798
599 619 272 283
1458 1478 285 296
111 131 804 818
997 1017 87 101
894 914 498 518
635 655 923 943
842 862 887 907
1101 1121 500 520
1294 1314 807 818
1531 1551 365 376
